In [1]:
import torch
import torch.nn as nn
from models.yolo import Model 

In [2]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, outputs, targets):
        loss = torch.mean((outputs - targets)**2)
        return loss
    
config_path = 'models/yolov5s.yaml'

model = Model(config_path)
model.load_state_dict(torch.load('yolov5s.pt')['model'].state_dict())  # yolov5s.ptは、学習済みの重みファイル


                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1   1182720  

<All keys matched successfully>

In [3]:
import yaml
import zipfile
import requests
import io

# 最初だけ
# # load config(coco128)
# config_path = 'data/coco128.yaml'
# with open(config_path) as f:
#     cf_data = yaml.safe_load(f)

# # downloa data
# data_path = './data/'
# response = requests.get(cf_data['download'])
# response.raise_for_status()
# # unzip
# with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
#     zip_ref.extractall('./data/')


In [6]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import json

class CustomDataset(Dataset):
    def __init__(self, img_dir, annotation_dir, transform=None):
        self.img_dir = img_dir
        self.annotation_dir = annotation_dir
        # デフォルトの変換にToTensorを追加
        self.transform = transforms.Compose([
            transforms.Resize((576, 576)),
            transforms.ToTensor(),
        ]) if transform is None else transform
        
        self.img_files = os.listdir(img_dir)

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, index):
        # 画像を読み込む
        img_path = os.path.join(self.img_dir, self.img_files[index])
        image = Image.open(img_path).convert("RGB")

        # アノテーションファイルを読み込む
        annotation_file = self.img_files[index].replace('.jpg', '.txt').replace('.png', '.txt')
        annotation_dir = os.path.join(self.annotation_dir, annotation_file)

        boxes = []
        with open(annotation_dir, 'r') as file:
            for line in file.readlines():
                class_label, x_center, y_center, width, height = [
                    float(x) for x in line.replace('\n', '').split()
                ]
                boxes.append([class_label, x_center, y_center, width, height])

        boxes = torch.tensor(boxes)

        if self.transform:
            image = self.transform(image)

        return image, boxes

def custom_collate_fn(batch):
    images = [item[0] for item in batch]
    bboxes = [item[1] for item in batch]

    images = torch.stack(images, 0) # 画像はバッチを作成

    return images, bboxes


img_dir = './data/coco128/images/train2017'
annotation_dir = './data/coco128/labels/train2017'
dataset = CustomDataset(
    img_dir=img_dir,
    annotation_dir=annotation_dir,
)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=custom_collate_fn)

for images, boxes in dataloader:
    print(boxes)

[tensor([[3.00000, 0.46548, 0.64553, 0.84824, 0.54459],
        [0.00000, 0.52266, 0.44012, 0.53712, 0.64162],
        [0.00000, 0.37864, 0.30618, 0.63771, 0.30642]]), tensor([[14.00000,  0.82078,  0.56129,  0.21863,  0.35270],
        [14.00000,  0.29346,  0.37634,  0.15962,  0.33484],
        [14.00000,  0.52598,  0.41653,  0.16667,  0.32554],
        [14.00000,  0.48671,  0.66271,  0.16165,  0.24138],
        [14.00000,  0.26703,  0.79969,  0.15380,  0.32250],
        [14.00000,  0.13952,  0.17415,  0.15917,  0.26742],
        [14.00000,  0.29550,  0.60918,  0.19040,  0.26864],
        [14.00000,  0.85972,  0.79617,  0.15707,  0.30918]]), tensor([[40.00000,  0.79792,  0.07959,  0.15303,  0.14573],
        [41.00000,  0.93908,  0.20101,  0.12184,  0.40202],
        [53.00000,  0.44251,  0.50581,  0.88502,  0.95438],
        [60.00000,  0.50000,  0.50112,  1.00000,  0.99775]]), tensor([[5.80000e+01, 1.78391e-01, 5.04327e-01, 1.20261e-01, 1.99808e-01],
        [7.10000e+01, 2.97717e-01

In [7]:
from utils.metrics import bbox_iou
from utils.torch_utils import de_parallel

class CustomLoss:
    sort_obj_iou = False

    # Compute losses
    def __init__(self, model, autobalance=False):
        device = next(model.parameters()).device  # get model device
        h = model.hyp  # hyperparameters

        # Define criteria
        BCEcls = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([h['cls_pw']], device=device))
        BCEobj = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([h['obj_pw']], device=device))

        # Class label smoothing https://arxiv.org/pdf/1902.04103.pdf eqn 3
        self.cp, self.cn = smooth_BCE(eps=h.get('label_smoothing', 0.0))  # positive, negative BCE targets

        # Focal loss
        g = h['fl_gamma']  # focal loss gamma
        if g > 0:
            BCEcls, BCEobj = FocalLoss(BCEcls, g), FocalLoss(BCEobj, g)

        m = de_parallel(model).model[-1]  # Detect() module
        self.balance = {3: [4.0, 1.0, 0.4]}.get(m.nl, [4.0, 1.0, 0.25, 0.06, 0.02])  # P3-P7
        self.ssi = list(m.stride).index(16) if autobalance else 0  # stride 16 index
        self.BCEcls, self.BCEobj, self.gr, self.hyp, self.autobalance = BCEcls, BCEobj, 1.0, h, autobalance
        self.na = m.na  # number of anchors
        self.nc = m.nc  # number of classes
        self.nl = m.nl  # number of layers
        self.anchors = m.anchors
        self.device = device

    def __call__(self, p, targets):  # predictions, targets
        lcls = torch.zeros(1, device=self.device)  # class loss
        lbox = torch.zeros(1, device=self.device)  # box loss
        lobj = torch.zeros(1, device=self.device)  # object loss
        tcls, tbox, indices, anchors = self.build_targets(p, targets)  # targets

        # Losses
        for i, pi in enumerate(p):  # layer index, layer predictions
            b, a, gj, gi = indices[i]  # image, anchor, gridy, gridx
            tobj = torch.zeros(pi.shape[:4], dtype=pi.dtype, device=self.device)  # target obj

            n = b.shape[0]  # number of targets
            if n:
                # pxy, pwh, _, pcls = pi[b, a, gj, gi].tensor_split((2, 4, 5), dim=1)  # faster, requires torch 1.8.0
                pxy, pwh, _, pcls = pi[b, a, gj, gi].split((2, 2, 1, self.nc), 1)  # target-subset of predictions

                # Regression
                pxy = pxy.sigmoid() * 2 - 0.5
                pwh = (pwh.sigmoid() * 2) ** 2 * anchors[i]
                pbox = torch.cat((pxy, pwh), 1)  # predicted box
                iou = bbox_iou(pbox, tbox[i], CIoU=True).squeeze()  # iou(prediction, target)
                lbox += (1.0 - iou).mean()  # iou loss

                # Objectness
                iou = iou.detach().clamp(0).type(tobj.dtype)
                if self.sort_obj_iou:
                    j = iou.argsort()
                    b, a, gj, gi, iou = b[j], a[j], gj[j], gi[j], iou[j]
                if self.gr < 1:
                    iou = (1.0 - self.gr) + self.gr * iou
                tobj[b, a, gj, gi] = iou  # iou ratio

                # Classification
                if self.nc > 1:  # cls loss (only if multiple classes)
                    t = torch.full_like(pcls, self.cn, device=self.device)  # targets
                    t[range(n), tcls[i]] = self.cp
                    lcls += self.BCEcls(pcls, t)  # BCE

                # Append targets to text file
                # with open('targets.txt', 'a') as file:
                #     [file.write('%11.5g ' * 4 % tuple(x) + '\n') for x in torch.cat((txy[i], twh[i]), 1)]

            obji = self.BCEobj(pi[..., 4], tobj)
            lobj += obji * self.balance[i]  # obj loss
            if self.autobalance:
                self.balance[i] = self.balance[i] * 0.9999 + 0.0001 / obji.detach().item()

        if self.autobalance:
            self.balance = [x / self.balance[self.ssi] for x in self.balance]
        lbox *= self.hyp['box']
        lobj *= self.hyp['obj']
        lcls *= self.hyp['cls']
        bs = tobj.shape[0]  # batch size

        return (lbox + lobj + lcls) * bs, torch.cat((lbox, lobj, lcls)).detach()

    def build_targets(self, p, targets):
        # Build targets for compute_loss(), input targets(image,class,x,y,w,h)
        na, nt = self.na, targets.shape[0]  # number of anchors, targets
        tcls, tbox, indices, anch = [], [], [], []
        gain = torch.ones(7, device=self.device)  # normalized to gridspace gain
        ai = torch.arange(na, device=self.device).float().view(na, 1).repeat(1, nt)  # same as .repeat_interleave(nt)
        targets = torch.cat((targets.repeat(na, 1, 1), ai[..., None]), 2)  # append anchor indices

        g = 0.5  # bias
        off = torch.tensor(
            [
                [0, 0],
                [1, 0],
                [0, 1],
                [-1, 0],
                [0, -1],  # j,k,l,m
                # [1, 1], [1, -1], [-1, 1], [-1, -1],  # jk,jm,lk,lm
            ],
            device=self.device).float() * g  # offsets

        for i in range(self.nl):
            anchors, shape = self.anchors[i], p[i].shape
            gain[2:6] = torch.tensor(shape)[[3, 2, 3, 2]]  # xyxy gain

            # Match targets to anchors
            t = targets * gain  # shape(3,n,7)
            if nt:
                # Matches
                r = t[..., 4:6] / anchors[:, None]  # wh ratio
                j = torch.max(r, 1 / r).max(2)[0] < self.hyp['anchor_t']  # compare
                # j = wh_iou(anchors, t[:, 4:6]) > model.hyp['iou_t']  # iou(3,n)=wh_iou(anchors(3,2), gwh(n,2))
                t = t[j]  # filter

                # Offsets
                gxy = t[:, 2:4]  # grid xy
                gxi = gain[[2, 3]] - gxy  # inverse
                j, k = ((gxy % 1 < g) & (gxy > 1)).T
                l, m = ((gxi % 1 < g) & (gxi > 1)).T
                j = torch.stack((torch.ones_like(j), j, k, l, m))
                t = t.repeat((5, 1, 1))[j]
                offsets = (torch.zeros_like(gxy)[None] + off[:, None])[j]
            else:
                t = targets[0]
                offsets = 0

            # Define
            bc, gxy, gwh, a = t.chunk(4, 1)  # (image, class), grid xy, grid wh, anchors
            a, (b, c) = a.long().view(-1), bc.long().T  # anchors, image, class
            gij = (gxy - offsets).long()
            gi, gj = gij.T  # grid indices

            # Append
            indices.append((b, a, gj.clamp_(0, shape[2] - 1), gi.clamp_(0, shape[3] - 1)))  # image, anchor, grid
            tbox.append(torch.cat((gxy - gij, gwh), 1))  # box
            anch.append(anchors[a])  # anchors
            tcls.append(c)  # class

        return tcls, tbox, indices, anch


In [10]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu
